# The net impact of electric vehicles
---

Electric vehicles are surging in popularity. However, electricity generation remains reliant on fossil fuels in many parts of the world. What does this mean for carbon emissions? 

In [40]:
#import packages
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import json
import os
import glob
import re

#MySQL user password
pw = "password"

### 1. MySQL Connector setup

In [41]:
#function for connecting to MySQL server
def create_server_connection(host_name, user_name, user_password):
    connect = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err: 
        print(f"Error: '{err}'")
        
    return connection

#create MySQL Server database
def create_database(connection, query):
    cursor = connection.cursor() #cursor object provides access to MySQL Server terminal methods
    try:
        cursor.execute(query) #execute 
        print('Database created successfully')
    except Error as err:
        print(f"Error: '{err}'")

#function for connecting to MySQL database
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
        
    return connection

#function for executing SQL queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit() #ensures query commands are implemented
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
#function for reading data from existing db
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")
        
#function for inserting a python list into the db
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val) #sql represents an SQL command, val represents list of values
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
#transform df into list of values
def df_to_list(df, columns):
    data = df[columns]
    df_list = data.values.tolist()
    return df_list

In [42]:
#establish connection
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [43]:
#create database
create_database_query = "CREATE DATABASE ev_net_impact"
create_database(connection, create_database_query)

db = "ev_net_impact"

Database created successfully


### 2. Load data

In [44]:
#load json files from folder into table format
def jsons_to_df(path):
    path_to_json = path
    json_pattern = os.path.join(path_to_json,'*.json')
    file_list = glob.glob(json_pattern)
    
    data = []
    for file in file_list:
        i = pd.read_json(file)
        data.append(i)
    
    data = pd.concat(data, ignore_index=True)
    return data

In [45]:
electricity_prod_data = pd.read_csv('electricity-production-by-source.csv')
ev_efficiency_data = pd.read_csv('ev_efficiency.csv', header=None, names=['Vehicle', 'Efficiency Wh/km', 'Segment'])
ev_sales_data = pd.read_csv('IEA-EV-dataEV-salesCarsHistorical.csv')
ev_share_data = pd.read_csv('IEA-EV-dataEV-sales-shareCarsHistorical.csv')
fuel_economy_data = jsons_to_df(path = 'fuel_economy').sort_values(["Country", "Year", "Product"], ascending=(True, True, True)).reset_index(drop=True)

### 3. Preprocess data with Pandas

In [46]:
def null_to_na(df, columns):
    df[columns] = df[columns].replace(np.nan, 'N/A')
    return df

def null_to_zero(df, columns):
    df[columns] = df[columns].replace(np.nan, 0)
    return df

def remove_string(df, columns, string):
    df[columns] = df[columns].str.replace(string, '')
    return df

def preprocess_electricity_data(data):
    data = null_to_na(data, 'Code')
    columns = ['Electricity from coal (TWh)',
               'Electricity from gas (TWh)',
               'Electricity from nuclear (TWh)',
               'Electricity from hydro (TWh)',
               'Electricity from other renewables (TWh)',
               'Electricity from solar (TWh)',
               'Electricity from oil (TWh)',
               'Electricity from wind (TWh)']
    data = null_to_zero(data, columns)
    return data

def preprocess_efficiency_data(data):
    data = remove_string(data, 'Efficiency Wh/km', ' Wh/km')
    return data

In [143]:
electricity_prod_data = preprocess_electricity_data(electricity_prod_data)
ev_efficiency_data = preprocess_efficiency_data(ev_efficiency_data)
ev_sales_data = null_to_na(ev_sales_data, 'unit')

### 3. Create MySQL tables

**3A Electricity production table** (Our World in Data)

In [75]:
create_electricity_production_table = """
CREATE TABLE electricity_production (
    region VARCHAR(80) NOT NULL,
    year INT NOT NULL,
    coal_twh FLOAT,
    gas_twh FLOAT,
    nuclear_twh FLOAT,
    hydro_twh FLOAT,
    other_renewables_twh FLOAT,
    solar_twh FLOAT,
    oil_twh FLOAT,
    wind_twh FLOAT,
    PRIMARY KEY (region, year)
); 
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_electricity_production_table)

MySQL Database connection successful
Query successful


In [78]:
columns = ['Entity', 'Year', 'Electricity from coal (TWh)', 'Electricity from gas (TWh)', 'Electricity from nuclear (TWh)', 'Electricity from hydro (TWh)', 'Electricity from other renewables (TWh)', 'Electricity from solar (TWh)', 'Electricity from oil (TWh)', 'Electricity from wind (TWh)']
sql = """
INSERT INTO electricity_production (region, year, coal_twh, gas_twh, nuclear_twh, hydro_twh, other_renewables_twh, solar_twh, oil_twh, wind_twh)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
val = df_to_list(electricity_prod_data, columns)

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql, val)

MySQL Database connection successful
Query successful


**3B Geo-type table**

In [80]:
create_geo_key_table = """
CREATE TABLE geo_types (
    region VARCHAR(80) PRIMARY KEY
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_geo_key_table)

MySQL Database connection successful
Query successful


In [82]:
insert_unique_regions = """
INSERT INTO geo_types (region)
SELECT DISTINCT region
FROM electricity_production;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, insert_unique_regions)

MySQL Database connection successful
Query successful


In [84]:
add_geo_types = """
ALTER TABLE geo_types
ADD geo_type VARCHAR(20);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, add_geo_types)

MySQL Database connection successful
Query successful


In [86]:
insert_geo_types = """
UPDATE geo_types
SET geo_type =
    CASE
        WHEN region = 'World' THEN 'World'
        WHEN region IN (
            'CIS', 
            'Central America', 
            'EU-27', 
            'EU27+1', 
            'Eastern Africa',
            'Europe (other)', 
            'Middle Africa',
            'Other Asia & Pacific',
            'Other CIS',
            'Other Caribbean', 
            'Other Middle East', 
            'Other Northern Africa',
            'Other South America', 
            'Other Southern Africa',
            'Western Africa',
            'Western Sahara'
            ) THEN 'Other'
        WHEN region IN (
            'Africa', 
            'Antarctica', 
            'Asia Pacific', 
            'North America', 
            'South & Central America', 
            'Europe', 
            'Middle East'
            ) THEN 'Continents'
        ELSE 'Countries'
    END;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, insert_geo_types)

MySQL Database connection successful
Query successful


**3C Emissions factors table** (IPCC)

In [170]:
create_emissions_factors_table = """
CREATE TABLE emissions_factors (
    source VARCHAR(20) PRIMARY KEY,
    metric VARCHAR(20) NOT NULL,
    factor INT NOT NULL
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_emissions_factors_table)

MySQL Database connection successful
Query successful


In [172]:
#factor value = grams of CO2e per kilowatt-hour
insert_factor_values = """
INSERT INTO emissions_factors
VALUES 
    ('battery_discharge', 'CO2e grams/kWh', 301),
    ('biomass', 'CO2e grams/kWh', 230),
    ('coal', 'CO2e grams/kWh', 820),
    ('gas', 'CO2e grams/kWh', 490),
    ('geothermal', 'CO2e grams/kWh', 38),
    ('hydro', 'CO2e grams/kWh', 24),
    ('nuclear', 'CO2e grams/kWh', 12),
    ('oil', 'CO2e grams/kWh', 650),
    ('solar', 'CO2e grams/kWh', 45),
    ('wind', 'CO2e grams/kWh', 11),
    ('other_renewables', 'CO2e grams/kWh', 301),
    ('petrol', 'CO2e grams/l', 2420);    
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, insert_factor_values)

MySQL Database connection successful
Query successful


**3D EV efficiency table** (Electric Vehicle Database)

In [93]:
create_ev_efficiency_table = """
CREATE TABLE ev_efficiency (
    id INT PRIMARY KEY AUTO_INCREMENT,
    vehicle VARCHAR(80) NOT NULL,
    wh_per_km FLOAT NOT NULL,
    segment VARCHAR(20) NOT NULL
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_ev_efficiency_table)

MySQL Database connection successful
Query successful


In [94]:
#insert DataFrame values into generators table
columns = ['Vehicle', 'Efficiency Wh/km', 'Segment']
sql = """
INSERT INTO ev_efficiency (vehicle, wh_per_km, segment)
VALUES (%s, %s, %s);
"""
val = df_to_list(ev_efficiency_data, columns)

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql, val)

MySQL Database connection successful
Query successful


In [95]:
#drop duplicate rows on vehicle and wh_per_km
duplicate_rows = """
SELECT vehicle, wh_per_km,
COUNT(*) AS count
FROM ev_efficiency
GROUP BY vehicle, wh_per_km
HAVING COUNT(*) > 1;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, duplicate_rows)

for result in results:
    print(result)

MySQL Database connection successful
('Audi e-tron 55 quattro', 237.0, 2)
('Audi e-tron Sportback 55 quattro', 231.0, 2)


In [96]:
drop_duplicate_rows = """
DELETE t1 
FROM ev_efficiency t1
INNER JOIN ev_efficiency t2
WHERE t1.id < t2.id AND t1.vehicle = t2.vehicle AND t1.wh_per_km = t2.wh_per_km;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, drop_duplicate_rows)

MySQL Database connection successful
Query successful


In [97]:
#drop id column
drop_id = """
ALTER TABLE ev_efficiency
DROP COLUMN id;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, drop_id)

MySQL Database connection successful
Query successful


In [98]:
#make vehicle column the primary key
new_pk = """
ALTER TABLE ev_efficiency
ADD PRIMARY KEY (vehicle);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, new_pk)

MySQL Database connection successful
Query successful


**3E Fuel economy table** (GFEI)

In [101]:
#create SQL table
create_fuel_economy_table = """
CREATE TABLE fuel_economy (
    region VARCHAR(40) NOT NULL,
    year INT NOT NULL,
    product VARCHAR(40) NOT NULL,
    lge_per_100km FLOAT NOT NULL,
    PRIMARY KEY (region, year, product)
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_fuel_economy_table)

MySQL Database connection successful
Query successful


In [102]:
#insert DataFrame values into table
columns = ['Country', 'Year', 'Product', 'Value']
sql = """
INSERT INTO fuel_economy (region, year, product, lge_per_100km)
VALUES (%s, %s, %s, %s);
"""
val = df_to_list(fuel_economy_data, columns)

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql, val)

MySQL Database connection successful
Query successful


In [156]:
#rename 'Korea' to 'South Korea'
change_korea_name = """
UPDATE fuel_economy
SET region = 'South Korea'
WHERE region = 'Korea';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, change_korea_name)

MySQL Database connection successful
Query successful


In [154]:
#rename 'USA' to 'United States'
change_us_name = """
UPDATE fuel_economy
SET region = 'United States'
WHERE region = 'USA';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, change_us_name)

MySQL Database connection successful
Query successful


**3F Vehicle segments table**

In [124]:
create_vehicle_segments_table = """
CREATE TABLE vehicle_segments (
    product VARCHAR(40) PRIMARY KEY
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_vehicle_segments_table)

MySQL Database connection successful
Query successful


In [125]:
insert_unique_products = """
INSERT INTO vehicle_segments (product)
SELECT DISTINCT product
FROM fuel_economy;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, insert_unique_products)

MySQL Database connection successful
Query successful


In [126]:
add_vehicle_segments = """
ALTER TABLE vehicle_segments
ADD segment VARCHAR(40);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, add_vehicle_segments)

MySQL Database connection successful
Query successful


In [127]:
insert_vehicle_segments = """
UPDATE vehicle_segments
SET segment =
    CASE
        WHEN product = 'Van/LCV' THEN 'spv'
        WHEN product = 'Medium car' THEN 'sedan'
        WHEN product = 'Large Car' THEN 'sedan'
        WHEN product = 'City car' THEN 'hatchback'
        WHEN product = 'Small SUV/Pick-up' THEN 'suv'
        WHEN product = 'Large SUV/Pick-up' THEN 'suv'
        ELSE product
    END;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, insert_vehicle_segments)

MySQL Database connection successful
Query successful


**3H Vehicle sales table** (IEA)

In [147]:
create_vehicle_sales_table = """
CREATE TABLE vehicle_sales (
    region VARCHAR(40) NOT NULL,
    parameter VARCHAR(20) NOT NULL,
    powertrain VARCHAR(10) NOT NULL,
    year INT NOT NULL,
    unit VARCHAR(20),
    value FLOAT NOT NULL,
    PRIMARY KEY (region, year, parameter, powertrain)
);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, create_vehicle_sales_table)

MySQL Database connection successful
Query successful


In [148]:
columns = ['region', 'parameter', 'powertrain', 'year', 'unit', 'value']
sql = """
INSERT INTO vehicle_sales (region, parameter, powertrain, year, unit, value)
VALUES (%s, %s, %s, %s, %s, %s)
"""
val = df_to_list(pd.concat([ev_sales_data[columns], ev_share_data[columns]]), columns)

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql, val)

MySQL Database connection successful
Query successful


In [158]:
#rename 'Korea' to 'South Korea'
change_korea_name = """
UPDATE vehicle_sales
SET region = 'South Korea'
WHERE region = 'Korea';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, change_korea_name)

MySQL Database connection successful
Query successful


In [159]:
#rename 'USA' to 'United States'
change_us_name = """
UPDATE vehicle_sales
SET region = 'United States'
WHERE region = 'USA';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, change_us_name)

MySQL Database connection successful
Query successful


In [166]:
#drop 'Other Europe'
drop_other_europe = """
DELETE FROM vehicle_sales
WHERE region = 'Other Europe';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, drop_other_europe)

MySQL Database connection successful
Query successful


**3I Establish foreign key relationships**

In [149]:
electricity_prod_region_fk = """
ALTER TABLE electricity_production
ADD CONSTRAINT electricity_prod_region_fk
FOREIGN KEY (region) 
REFERENCES geo_types(region);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, electricity_prod_region_fk)

MySQL Database connection successful
Query successful


In [150]:
fuel_economy_region_fk = """
ALTER TABLE fuel_economy
ADD CONSTRAINT fuel_economy_region_fk
FOREIGN KEY (region) 
REFERENCES geo_types(region);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, fuel_economy_region_fk)

MySQL Database connection successful
Query successful


In [167]:
vehicle_sales_region_fk = """
ALTER TABLE vehicle_sales
ADD CONSTRAINT vehicle_sales_region_fk
FOREIGN KEY (region) 
REFERENCES geo_types(region);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, vehicle_sales_region_fk)

MySQL Database connection successful
Query successful


In [165]:
fuel_economy_product_fk = """
ALTER TABLE fuel_economy
ADD CONSTRAINT fuel_economy_product_fk
FOREIGN KEY (product) 
REFERENCES vehicle_segments(product);
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, fuel_economy_product_fk)

MySQL Database connection successful
Query successful


### 4. Views

In [213]:
#total emissions
emissions_view = """
CREATE VIEW total_emissions AS
SELECT region, year, (
    coal_twh * (SELECT factor FROM emissions_factors WHERE source = 'coal') + 
    gas_twh * (SELECT factor FROM emissions_factors WHERE source = 'gas') + 
    nuclear_twh * (SELECT factor FROM emissions_factors WHERE source = 'nuclear') + 
    hydro_twh * (SELECT factor FROM emissions_factors WHERE source = 'hydro') + 
    other_renewables_twh * (SELECT factor FROM emissions_factors WHERE source = 'other_renewables') + 
    solar_twh * (SELECT factor FROM emissions_factors WHERE source = 'solar') + 
    oil_twh * (SELECT factor FROM emissions_factors WHERE source = 'oil') + 
    wind_twh * (SELECT factor FROM emissions_factors WHERE source = 'wind')
    ) AS total_emissions
FROM electricity_production;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, emissions_view)

MySQL Database connection successful
Query successful


In [208]:
#total twh
total_twh_view = """
CREATE VIEW total_twh AS
SELECT region, year, (
    coal_twh + gas_twh + nuclear_twh + hydro_twh + other_renewables_twh + solar_twh + oil_twh + wind_twh
    ) AS total_twh
FROM electricity_production;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, total_twh_view)

MySQL Database connection successful
Query successful


In [219]:
#electric CO2e per km
ev_emissions_view = """
CREATE VIEW ev_co2_per_km AS
SELECT t1.region, t1.year, geo_type, AVG(total_emissions / total_twh / 1000 * wh_per_km) AS CO2e_grams_per_km_electric
FROM electricity_production t1
INNER JOIN total_emissions t2 ON t1.region = t2.region AND t1.year = t2.year
INNER JOIN total_twh t3 ON t1.region = t3.region AND t1.year = t3.year
INNER JOIN ev_efficiency t4
INNER JOIN geo_types t5 ON t1.region = t5.region
WHERE t1.year = 2019 AND geo_type = 'Countries'
GROUP BY t1.region
ORDER BY 1;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, ev_emissions_view)

MySQL Database connection successful
Query successful


In [223]:
#petrol CO2e per km
petrol_emissions_view = """
CREATE VIEW petrol_co2_per_km AS
SELECT region, year, (lge_per_100km * (SELECT factor FROM emissions_factors WHERE source = 'petrol') / 100) AS CO2e_grams_per_km_petrol
FROM fuel_economy
WHERE year = 2019 AND product = 'Petrol';
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, petrol_emissions_view)

MySQL Database connection successful
Query successful


In [236]:
#annual ev sales
annual_ev_sales_view = """
CREATE VIEW annual_ev_sales AS
SELECT region, year, SUM(value) AS ev_sales
FROM vehicle_sales
WHERE parameter = 'EV sales'
GROUP BY region, year;
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, annual_ev_sales_view)

MySQL Database connection successful
Query successful


In [244]:
#total vehicle sales
annual_vehicle_sales_view = """
CREATE VIEW annual_vehicle_sales AS
SELECT t1.region, t1.year, 
    ev_sales / (value / 100) AS total_sales
FROM annual_ev_sales t1
INNER JOIN vehicle_sales t2 ON t1.region = t2.region AND t1.year = t2.year
WHERE parameter = 'EV sales share'; 
"""

connection = create_db_connection("localhost", "root", pw, db)
execute_query(connection, annual_vehicle_sales_view)

MySQL Database connection successful
Query successful


### 5. Queries

In [214]:
#query 1: total emissions by country by year (CO2e tonnes*1000)
q1 = """
SELECT t1.region, geo_type, year, total_emissions
FROM total_emissions t1
INNER JOIN geo_types t2 ON t1.region = t2.region
WHERE geo_type != 'Other';
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q1)

for result in results:
    print(result)

MySQL Database connection successful
('Afghanistan', 'Countries', 2000, 83.43800073862076)
('Afghanistan', 'Countries', 2001, 58.01199832558632)
('Afghanistan', 'Countries', 2002, 77.99999982118607)
('Afghanistan', 'Countries', 2003, 166.52999275922775)
('Afghanistan', 'Countries', 2004, 174.76999807357788)
('Afghanistan', 'Countries', 2005, 182.72000509500504)
('Afghanistan', 'Countries', 2006, 113.28800204396248)
('Afghanistan', 'Countries', 2007, 114.97200009226799)
('Afghanistan', 'Countries', 2008, 104.14811205826481)
('Afghanistan', 'Countries', 2009, 98.00510078293883)
('Afghanistan', 'Countries', 2010, 108.68410085900541)
('Afghanistan', 'Countries', 2011, 137.77010267241712)
('Afghanistan', 'Countries', 2012, 103.55999707058072)
('Afghanistan', 'Countries', 2013, 128.68999785743654)
('Afghanistan', 'Countries', 2014, 100.13199953362346)
('Afghanistan', 'Countries', 2015, 96.06899807602167)
('Afghanistan', 'Countries', 2016, 99.67500235512853)
('Afghanistan', 'Countries', 2017,

('Nepal', 'Countries', 2001, 60.19399953261018)
('Nepal', 'Countries', 2002, 61.40200179070234)
('Nepal', 'Countries', 2003, 56.36000035237521)
('Nepal', 'Countries', 2004, 65.52200067788363)
('Nepal', 'Countries', 2005, 65.6580022200942)
('Nepal', 'Countries', 2006, 70.84199842065573)
('Nepal', 'Countries', 2007, 72.02400283142924)
('Nepal', 'Countries', 2008, 72.59400238841772)
('Nepal', 'Countries', 2009, 82.18700233101845)
('Nepal', 'Countries', 2010, 78.261212376412)
('Nepal', 'Countries', 2011, 84.64356815093197)
('Nepal', 'Countries', 2012, 96.15630218293518)
('Nepal', 'Countries', 2013, 90.13446211069822)
('Nepal', 'Countries', 2014, 91.44367829908151)
('Nepal', 'Countries', 2015, 82.95907459687442)
('Nepal', 'Countries', 2016, 101.61805908661336)
('Nepal', 'Countries', 2017, 113.62240429315716)
('Nepal', 'Countries', 2018, 120.02288522385061)
('Nepal', 'Countries', 2019, 131.57863447628915)
('Netherlands', 'Countries', 1985, 33184.73169982806)
('Netherlands', 'Countries', 1986

In [211]:
#query 2: emissions pct by source by country by year
q2 = """
SELECT t1.region, geo_type, t1.year,
    coal_twh / total_twh AS coal_pct,
    gas_twh / total_twh AS gasl_pct,
    nuclear_twh / total_twh AS nuclear_pct,
    hydro_twh / total_twh AS hydro_pct,
    other_renewables_twh / total_twh AS other_renewables_pct,
    solar_twh / total_twh AS solar_pct,
    oil_twh / total_twh AS oil_pct,
    wind_twh / total_twh AS wind_pct,
    (nuclear_twh + hydro_twh + other_renewables_twh + solar_twh + wind_twh) / total_twh AS renewables_pct   
FROM electricity_production t1
INNER JOIN geo_types t2 ON t1.region = t2.region
INNER JOIN total_twh t3 ON t1.region = t3.region AND t1.year = t3.year
WHERE geo_type != 'Other';
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q2)

for result in results:
    print(result)

MySQL Database connection successful
('Afghanistan', 'Countries', 2000, 0.0, 0.33190577871488813, 0.0, 0.6680942212851119, 0.0, 0.0, 0.0, 0.0, 0.6680942212851119)
('Afghanistan', 'Countries', 2001, 0.0, 0.15878378029727666, 0.0, 0.8412162197027233, 0.0, 0.0, 0.0, 0.0, 0.8412162197027233)
('Afghanistan', 'Countries', 2002, 0.0, 0.19213973514975666, 0.0, 0.8078602648502433, 0.0, 0.0, 0.0, 0.0, 0.8078602648502433)
('Afghanistan', 'Countries', 2003, 0.0, 0.3290734737076702, 0.0, 0.6709265262923299, 0.0, 0.0, 0.0, 0.0, 0.6709265262923299)
('Afghanistan', 'Countries', 2004, 0.0, 0.36800894683058966, 0.0, 0.6319910531694103, 0.0, 0.0, 0.0, 0.0, 0.6319910531694103)
('Afghanistan', 'Countries', 2005, 0.0, 0.3683083694207874, 0.0, 0.6316916305792126, 0.0, 0.0, 0.0, 0.0, 0.6316916305792126)
('Afghanistan', 'Countries', 2006, 0.0, 0.23894862181265375, 0.0, 0.7610513781873463, 0.0, 0.0, 0.0, 0.0, 0.7610513781873463)
('Afghanistan', 'Countries', 2007, 0.0, 0.20930231898763718, 0.0, 0.790697681012362

('Laos', 'Countries', 2015, 0.1324558148584009, 0.0, 0.0, 0.8670451663034517, 0.0002495094190736575, 0.0002495094190736575, 0.0, 0.0, 0.8675441851415991)
('Laos', 'Countries', 2016, 0.29912890316645646, 0.0, 0.0, 0.7004714205893253, 0.0001998381221090912, 0.0001998381221090912, 0.0, 0.0, 0.7008710968335435)
('Laos', 'Countries', 2017, 0.34009661423824633, 0.0, 0.0, 0.6589431640976144, 0.0008608883896019566, 9.933327453726012e-05, 0.0, 0.0, 0.6599033857617537)
('Laos', 'Countries', 2018, 0.3376065194431489, 0.0, 0.0, 0.660540775278916, 0.0013447054382760965, 0.0005079998396589964, 0.0, 0.0, 0.6623934805568511)
('Laos', 'Countries', 2019, 0.4251780805943868, 0.0, 0.0, 0.572492067276555, 0.0017375168401395, 0.000592335288918717, 0.0, 0.0, 0.5748219194056132)
('Latvia', 'Countries', 1985, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0)
('Latvia', 'Countries', 1986, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0)
('Latvia', 'Countries', 1987, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0)
('Latvia'

In [225]:
q3 = """
SELECT region, year, CO2e_grams_per_km_electric
FROM ev_co2_per_km;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q3)

for result in results:
    print(result)

MySQL Database connection successful
('Afghanistan', 2019, 18.801376132466455)
('Albania', 2019, 4.846582138095819)
('Algeria', 2019, 97.69533264303556)
('American Samoa', 2019, 130.77522935779817)
('Angola', 2019, 26.444611081785737)
('Antigua and Barbuda', 2019, 125.43135591094584)
('Argentina', 2019, 70.04898154609779)
('Armenia', 2019, 41.1661090966661)
('Aruba', 2019, 111.82351105824931)
('Australia', 2019, 117.14983011245299)
('Austria', 2019, 32.72915215100591)
('Azerbaijan', 2019, 106.6430474153477)
('Bahamas', 2019, 130.55272174526576)
('Bahrain', 2019, 98.58569069588184)
('Bangladesh', 2019, 105.56924374778673)
('Barbados', 2019, 125.97674479492618)
('Belarus', 2019, 96.50471674748341)
('Belgium', 2019, 38.50789769964141)
('Belize', 2019, 26.758113481746843)
('Benin', 2019, 105.86538028763366)
('Bermuda', 2019, 130.77522935779817)
('Bhutan', 2019, 4.829756473427495)
('Bolivia', 2019, 64.10132536440763)
('Bosnia and Herzegovina', 2019, 102.90330363726251)
('Botswana', 2019, 16

In [224]:
#query 4: carbon efficiency of petrol vehicles in 2019 by country
q4 = """
SELECT region, year, CO2e_grams_per_km_petrol
FROM petrol_co2_per_km;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q4)

for result in results:
    print(result)

MySQL Database connection successful
('Argentina', 2019, 183.19400415420532)
('Australia', 2019, 199.16598892211914)
('Brazil', 2019, 229.9)
('Canada', 2019, 212.23401107788087)
('Chile', 2019, 191.42199630737304)
('China', 2019, 180.0480013847351)
('France', 2019, 141.32800369262696)
('Germany', 2019, 161.8980013847351)
('India', 2019, 133.34200553894044)
('Italy', 2019, 148.3460027694702)
('Japan', 2019, 130.92199630737304)
('Malaysia', 2019, 166.98000230789185)
('Mexico', 2019, 162.13999538421632)
('Russia', 2019, 202.31199169158936)
('South Africa', 2019, 171.33599815368652)
('South Korea', 2019, 157.54200553894043)
('Turkey', 2019, 158.02600507736207)
('Ukraine', 2019, 192.1480013847351)
('United Kingdom', 2019, 162.13999538421632)
('United States', 2019, 208.8460027694702)


In [226]:
#query 5: CO2e_grams_per_km for petrol and electric PLUS difference by country in 2019
q5 = """
SELECT t1.region, t1.year, CO2e_grams_per_km_electric, CO2e_grams_per_km_petrol,
    (CO2e_grams_per_km_petrol - CO2e_grams_per_km_electric) AS absolute_diff,
    ((CO2e_grams_per_km_electric / CO2e_grams_per_km_petrol) - 1) AS percentage_diff
FROM ev_co2_per_km t1
INNER JOIN petrol_co2_per_km t2 ON t1.region = t2.region AND t1.year = t2.year;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q5)

for result in results:
    print(result)

MySQL Database connection successful
('Argentina', 2019, 70.04898154609779, 183.19400415420532, 113.14502260810752, -0.6176240490538468)
('Australia', 2019, 117.14983011245299, 199.16598892211914, 82.01615880966615, -0.4117980145783693)
('Brazil', 2019, 26.156537700511933, 229.9, 203.74346229948807, -0.8862264562831147)
('Canada', 2019, 27.485406009419204, 212.23401107788087, 184.74860506846167, -0.870494809621568)
('Chile', 2019, 79.07472854216084, 191.42199630737304, 112.3472677652122, -0.5869088711456767)
('China', 2019, 110.61437425939089, 180.0480013847351, 69.43362712534422, -0.385639532743133)
('France', 2019, 14.02585602845293, 141.32800369262696, 127.30214766417403, -0.9007567101920039)
('Germany', 2019, 73.5781778983078, 161.8980013847351, 88.3198234864273, -0.5455275712548402)
('India', 2019, 124.68837696719358, 133.34200553894044, 8.653628571746864, -0.06489799322254608)
('Italy', 2019, 72.04204508977722, 148.3460027694702, 76.30395767969299, -0.5143647705713338)
('Japan', 

In [228]:
#query 6: CO2e_grams_per_km for petrol and electric by segment in 2019
q6 = """
SELECT t1.segment, 
    AVG(lge_per_100km * (SELECT factor FROM emissions_factors WHERE source = 'petrol') / 100) AS CO2e_grams_per_km_petrol,
    AVG(wh_per_km * total_emissions / total_twh / 1000) AS CO2e_grams_per_km_electric
FROM ev_efficiency t1
INNER JOIN vehicle_segments t2 ON t1.segment = t2.segment
INNER JOIN fuel_economy t3 ON t2.product = t3.product
INNER JOIN total_emissions t4 ON t3.year = t4.year AND t3.region = t4.region
INNER JOIN total_twh t5 ON t4.year = t5.year AND t4.region = t5.region
WHERE t3.year = 2019 AND t1.segment IN ('spv', 'sedan', 'hatchback', 'suv')
GROUP BY t1.segment;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q6)

for result in results:
    print(result)

MySQL Database connection successful
('hatchback', 142.5160009860993, 65.95627772797005)
('sedan', 167.18742848350882, 70.96834592169351)
('suv', 201.12399885654642, 76.77852143205016)
('spv', 200.0460022449507, 102.75741971157998)


In [247]:
#query 7: emissions saved if all 2019 sales ev
q7 = """
SELECT t1.region, t1.year,
    (total_sales - ev_sales)
    * (CO2e_grams_per_km_petrol - CO2e_grams_per_km_electric)
    / 1000000 AS CO2_tonnes_saved_per_km
FROM annual_vehicle_sales t1
INNER JOIN annual_ev_sales t2 ON t1.year = t2.year AND t1.region = t2.region
INNER JOIN petrol_co2_per_km t3 ON t1.year = t3.year AND t1.region = t3.region
INNER JOIN ev_co2_per_km t4 ON t1.year = t4.year AND t1.region = t4.region
WHERE t1.year = 2019
ORDER BY 3 desc;
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q7)

for result in results:
    print(result)

MySQL Database connection successful
('United States', 2019, 2398.9795297877167)
('China', 2019, 1467.1229631823749)
('Brazil', 2019, 463.12702922401036)
('Germany', 2019, 319.223120088056)
('Canada', 2019, 280.45442195476085)
('France', 2019, 269.219793365771)
('United Kingdom', 2019, 248.71970338867882)
('Japan', 2019, 185.14652874797181)
('Italy', 2019, 145.07712594920696)
('South Korea', 2019, 103.21069034388539)
('Mexico', 2019, 83.77050620037075)
('Australia', 2019, 65.5565875851627)
('India', 2019, 26.553659864501626)
('Chile', 2019, 23.710836238541976)
('South Africa', 2019, 9.172427208387733)


In [265]:
#query 8: renewables pct of electricity production, total production and ev_co2e_per_km by country in 2019
q8 = """
SELECT t1.region, t1.year,
    (nuclear_twh + hydro_twh + other_renewables_twh + solar_twh + wind_twh) / total_twh AS renewables_pct,
    total_twh,
    CO2e_grams_per_km_electric
FROM electricity_production t1
INNER JOIN geo_types t2 ON t1.region = t2.region
INNER JOIN total_twh t3 ON t1.region = t3.region AND t1.year = t3.year
INNER JOIN ev_co2_per_km t4 ON t1.region = t4.region AND t1.year = t4.year
WHERE t2.geo_type != 'Other';
"""

connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q8)

for result in results:
    print(result)

MySQL Database connection successful
('Afghanistan', 2019, 0.8527131784624703, 1.0319999679923058, 18.801376132466455)
('Albania', 2019, 1.0, 5.175940200686455, 4.846582138095819)
('Algeria', 2019, 0.009851479608411208, 74.34822370111942, 97.69533264303556)
('American Samoa', 2019, 0.0, 0.16300000250339508, 130.77522935779817)
('Angola', 2019, 0.8110872531181763, 12.477753572165966, 26.444611081785737)
('Antigua and Barbuda', 2019, 0.043902439822191885, 0.3416666528210044, 125.43135591094584)
('Argentina', 2019, 0.32244535569016913, 132.68260765075684, 70.04898154609779)
('Armenia', 2019, 0.605292928870082, 7.256470171269029, 41.1661090966661)
('Aruba', 2019, 0.1479878596436955, 0.9913786668330431, 111.82351105824931)
('Australia', 2019, 0.21444372729339392, 253.6789026260376, 117.14983011245299)
('Austria', 2019, 0.7696641838590745, 71.62575209140778, 32.72915215100591)
('Azerbaijan', 2019, 0.0765850607026834, 24.595963642001152, 106.6430474153477)
('Bahamas', 2019, 0.0018280049417975

### 7. Export to CSV

In [266]:
queries = [q1, q2, q3, q4, q5, q6, q7, q8]

for (i, q) in enumerate(queries):
    df = pd.read_sql_query(q, connection)
    df = pd.DataFrame(df)
    df.to_csv('/home/lachy/dml/ev-net-impact/sql_queries/q' + str(i + 1) + '.csv', index=False)